In [1]:

import pandas as pd
import numpy as np 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [5]:
keyword_df = pd.read_csv('IT2_Keywords_granular.csv')

In [6]:
keyword_df.head()

,Unnamed: 0,keyword,Score,POS,query,time
0,0,adjustments,0.157,Noun,Technology Project Manager,2020-02-09 17:57:52.192017
1,1,updates consistent,0.100,Noun,Technology Project Manager,2020-02-09 17:57:52.192017
2,2,type renewal,0.100,Noun,Technology Project Manager,2020-02-09 17:57:52.192017
3,3,track costs,0.100,Noun,Technology Project Manager,2020-02-09 17:57:52.192017
4,4,tools methodologies,0.100,Noun,Technology Project Manager,2020-02-09 17:57:52.192017


In [7]:
keyword_df["query"].unique()

array(['Technology Project Manager', 'Technology Product Manager',
       'Data Scientist', 'DevOps Engineer', 'Software Engineer',
       'Data Engineer', 'Solutions Architect', 'Data Analyst',
       'Full Stack Developer', 'Development Manager', 'CTO', 'CIO',
       'Security Engineer', 'Mobile Application Developer',
       'Senior Web Developer', 'Cloud Solutions Architect',
       'Information Technology Manager', 'Applications Architect',
       'Big data engineer', 'Information systems security manager',
       'Data security analyst'], dtype=object)

In [9]:
salary_df = pd.read_csv('job_avg_salaries.csv')

In [10]:
salary_df.head()

,Job_Tittle,Domain,Date,Avg_Salary
0,Technology Project Manager,Information Technology,2020-02-09,103175
1,Technology Product Manager,Information Technology,2020-02-09,106838
2,Data Scientist,Information Technology,2020-02-09,123253
3,DevOps Engineer,Information Technology,2020-02-09,126982
4,Software Engineer,Information Technology,2020-02-09,107254


In [11]:
keyword_df(['keyword', 'POS', 'time', 'query'], as_index=False)['Score'].agg({"keyword_count": "count"}).sort_values(['keyword_count'], ascending=False)

TypeError: 'DataFrame' object is not callable